In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from scipy.stats import pearsonr
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer import Predictive
from pyro.infer import MCMC, NUTS

In [ ]:
#import creep data
creep_df = pd.read_csv('SS316_PI_dataset.csv')
creep_df

In [ ]:
creep_df.head()

In [ ]:
class BNN(PyroModule):
    def __init__(self, in_dim=21, out_dim=1, hid_dim=10, n_hid_layers=5, prior_scale=5.):
        super().__init__()

        self.activation = nn.Tanh()  # could also be ReLU or LeakyReLU
        assert in_dim > 0 and out_dim > 0 and hid_dim > 0 and n_hid_layers > 0  # make sure the dimensions are valid

        # Define the layer sizes and the PyroModule layer list
        self.layer_sizes = [in_dim] + n_hid_layers * [hid_dim] + [out_dim]
        layer_list = [PyroModule[nn.Linear](self.layer_sizes[idx - 1], self.layer_sizes[idx]) for idx in
                      range(1, len(self.layer_sizes))]
        self.layers = PyroModule[torch.nn.ModuleList](layer_list)

        for layer_idx, layer in enumerate(self.layers):
            layer.weight = PyroSample(dist.Normal(0., prior_scale * np.sqrt(2 / self.layer_sizes[layer_idx])).expand(
                [self.layer_sizes[layer_idx + 1], self.layer_sizes[layer_idx]]).to_event(2))
            layer.bias = PyroSample(dist.Normal(0., prior_scale).expand([self.layer_sizes[layer_idx + 1]]).to_event(1))

    def forward(self, x, y=None):
        # x = x.reshape(-1, 1)
        x = self.activation(self.layers[0](x))  # input --> hidden
        for layer in self.layers[1:-1]:
            x = self.activation(layer(x))  # hidden --> hidden
        mu = self.layers[-1](x).squeeze()  # hidden --> output
        sigma = pyro.sample("sigma", dist.Gamma(.5, 1))  # infer the response noise

        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mu, sigma * sigma), obs=y)
        return mu

In [ ]:
rm_state = 123
test_size = 0.6

X, X_test, y, y_test = train_test_split(np.array(creep_df.iloc[:, 0:21]), np.array(creep_df.iloc[:,21]), shuffle=True, test_size=test_size, random_state=rm_state)

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

In [ ]:
idx = np.arange(len(y))

train_ratio = 0.1
X_train, _, y_train, _, idx_train, idx_pool = train_test_split(X, y, idx, train_size=train_ratio, shuffle=True, random_state=rm_state)

In [ ]:
pcc_variance = []
r2_variance = []
rmse_variance = []
mae_variance = []
num_training_data=[]

pcc_random = []
r2_random = []
num_iteration = []

X_train_var = X_train
X_train_ran = X_train
y_train_var = y_train
y_train_ran = y_train
idx_pool_var = idx_pool
idx_pool_ran = idx_pool
idx_train_var = idx_train
idx_train_ran = idx_train

In [ ]:
n_iter = 20
test_list = []

for i in range(n_iter):
    print(f"Performing iteration : {i}")

    if i != 0:
        # find 10 data points with the highest variance
        
        q_points_var = np.argpartition(y_pred_unc_pool_var, -10)[-10:]
        # indices of those points in idx_pool
        idx_pool_train_var = idx_pool_var[q_points_var]

        idx_train_var = np.append(idx_train_var, idx_pool_train_var)
        idx_pool_var = np.delete(idx_pool_var, q_points_var)
        
        X_train_var = X[idx_train_var]
        y_train_var = y[idx_train_var]
       
    print(f"Number of training data with variance: {len(idx_train_var)}")
    print(f"Number of pooling data with variance: {len(idx_pool_var)}")
    
    num_training_data.append(len(idx_train_var))
    
    model = BNN(hid_dim=10, n_hid_layers=3, prior_scale=1)
    nuts_kernel = NUTS(model, jit_compile=False)
    mcmc = MCMC(nuts_kernel, num_samples=100)
    mcmc.run(torch.Tensor(X_train_var), torch.Tensor(y_train_var))
    predictive_var = Predictive(model=model, posterior_samples=mcmc.get_samples())
    preds_var = predictive_var(torch.Tensor(X_test))

    # mean and standard deviation of the test dataset
    y_pred_test_var = preds_var['obs'].T.detach().numpy().mean(axis=1)
    y_std_test_var = preds_var['obs'].T.detach().numpy().std(axis=1)

    preds_pool_var = predictive_var(torch.Tensor(X[idx_pool_var]))
    y_pred_unc_pool_var = preds_pool_var['obs'].T.detach().numpy().std(axis=1)

    print('PCC_test variance', pearsonr(np.squeeze(y_test), np.squeeze(y_pred_test_var))[0])
    print('R2_test variance', r2_score(np.squeeze(y_test), np.squeeze(y_pred_test_var)))
    print('RMSE', np.sqrt(mean_squared_error(y_test, y_pred_test_var)))
    print('MAE', np.mean(abs(y_test - y_pred_test_var)))

    pcc_variance.append(pearsonr(np.squeeze(y_test), np.squeeze(y_pred_test_var))[0])
    r2_variance.append(r2_score(np.squeeze(y_test), np.squeeze(y_pred_test_var)))
    rmse_variance.append(np.sqrt(mean_squared_error(y_test, y_pred_test_var)))
    mae_variance.append(np.mean(abs(y_test - y_pred_test_var)))

In [ ]:
import pickle

with open('AL_BNN_MCMC_PI.pkl', 'wb') as f:

    pickle.dump({'train_numbs':num_training_data, 'pcc':pcc_variance,'r2':r2_variance, 'rsme': rmse_variance, 'mae': mae_variance}, f)
    f.close()

pkl_file = open('AL_BNN_MCMC_PI.pkl', 'rb')  
test_ALGPR = pickle.load(pkl_file)